In [2]:
pip install catboost

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/1c/e1/78e635a1e5f0066bd02a1ecfd658ad09fe30d275c65c2d0dd76fe253e648/catboost-1.2.7-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b6345cc143f671e1ed068818d5cf/graphviz-0.20.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB 108.9 kB/s eta 0:15:34
   ---------------------------------------- 0.0/101.7 MB 108.9 kB/s eta 0:15:34
   ---------------------------------------- 0.0/101.7 MB 140.3 kB/s eta 0:12:06
   ---------------------------------------- 0.0/101.7 MB 140.3 kB/s eta 0:12:06
   ---------------------------------------- 0.0/101.7 M

   -------------------------- ------------ 69.4/101.7 MB 262.3 kB/s eta 0:02:04
   -------------------------- ------------ 69.4/101.7 MB 262.3 kB/s eta 0:02:04
   -------------------------- ------------ 69.4/101.7 MB 262.3 kB/s eta 0:02:04
   -------------------------- ------------ 69.4/101.7 MB 262.3 kB/s eta 0:02:04
   -------------------------- ------------ 69.5/101.7 MB 263.6 kB/s eta 0:02:03
   -------------------------- ------------ 69.5/101.7 MB 263.6 kB/s eta 0:02:03
   -------------------------- ------------ 69.5/101.7 MB 263.6 kB/s eta 0:02:03
   -------------------------- ------------ 69.5/101.7 MB 263.6 kB/s eta 0:02:03
   -------------------------- ------------ 69.5/101.7 MB 263.6 kB/s eta 0:02:03
   -------------------------- ------------ 69.5/101.7 MB 263.6 kB/s eta 0:02:03
   -------------------------- ------------ 69.5/101.7 MB 263.6 kB/s eta 0:02:03
   -------------------------- ------------ 69.6/101.7 MB 266.8 kB/s eta 0:02:01
   -------------------------- ----------

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [10]:
train_dir = r'D:\FCI_Material\First\Computer Vision\sec4\Alzheimer_s Dataset\train'  
test_dir = r'D:\FCI_Material\First\Computer Vision\sec4\Alzheimer_s Dataset\test'

In [11]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [12]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [13]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [14]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [15]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

D:\program\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

D:\program\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 88ms/step


In [17]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step


In [18]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3222600	total: 195ms	remaining: 3m 14s
1:	learn: 1.2669202	total: 215ms	remaining: 1m 47s
2:	learn: 1.2210268	total: 236ms	remaining: 1m 18s
3:	learn: 1.1783602	total: 266ms	remaining: 1m 6s
4:	learn: 1.1438342	total: 291ms	remaining: 57.9s
5:	learn: 1.1132650	total: 322ms	remaining: 53.4s
6:	learn: 1.0855929	total: 358ms	remaining: 50.7s
7:	learn: 1.0625350	total: 392ms	remaining: 48.7s
8:	learn: 1.0421404	total: 429ms	remaining: 47.2s
9:	learn: 1.0232832	total: 481ms	remaining: 47.6s
10:	learn: 1.0052896	total: 515ms	remaining: 46.3s
11:	learn: 0.9893882	total: 550ms	remaining: 45.3s
12:	learn: 0.9759221	total: 585ms	remaining: 44.4s
13:	learn: 0.9649219	total: 618ms	remaining: 43.6s
14:	learn: 0.9551350	total: 654ms	remaining: 43s
15:	learn: 0.9444047	total: 699ms	remaining: 43s
16:	learn: 0.9340210	total: 734ms	remaining: 42.4s
17:	learn: 0.9254481	total: 762ms	remaining: 41.6s
18:	learn: 0.9165150	total: 790ms	remaining: 40.8s
19:	learn: 0.9089193	total: 840ms	remainin

165:	learn: 0.5102003	total: 6.79s	remaining: 34.1s
166:	learn: 0.5084922	total: 6.84s	remaining: 34.1s
167:	learn: 0.5068628	total: 6.89s	remaining: 34.1s
168:	learn: 0.5046924	total: 6.93s	remaining: 34.1s
169:	learn: 0.5035855	total: 6.96s	remaining: 34s
170:	learn: 0.5020314	total: 6.99s	remaining: 33.9s
171:	learn: 0.5003630	total: 7.02s	remaining: 33.8s
172:	learn: 0.4987091	total: 7.06s	remaining: 33.7s
173:	learn: 0.4971978	total: 7.09s	remaining: 33.6s
174:	learn: 0.4952140	total: 7.11s	remaining: 33.5s
175:	learn: 0.4929933	total: 7.15s	remaining: 33.5s
176:	learn: 0.4913966	total: 7.17s	remaining: 33.4s
177:	learn: 0.4896877	total: 7.21s	remaining: 33.3s
178:	learn: 0.4885062	total: 7.25s	remaining: 33.3s
179:	learn: 0.4867646	total: 7.27s	remaining: 33.1s
180:	learn: 0.4856661	total: 7.32s	remaining: 33.1s
181:	learn: 0.4837319	total: 7.38s	remaining: 33.1s
182:	learn: 0.4824507	total: 7.43s	remaining: 33.2s
183:	learn: 0.4810539	total: 7.47s	remaining: 33.1s
184:	learn: 0.

330:	learn: 0.3126234	total: 12.7s	remaining: 25.8s
331:	learn: 0.3120593	total: 12.8s	remaining: 25.7s
332:	learn: 0.3112132	total: 12.8s	remaining: 25.7s
333:	learn: 0.3106471	total: 12.8s	remaining: 25.6s
334:	learn: 0.3097447	total: 12.9s	remaining: 25.6s
335:	learn: 0.3093617	total: 12.9s	remaining: 25.5s
336:	learn: 0.3083230	total: 12.9s	remaining: 25.5s
337:	learn: 0.3073653	total: 13s	remaining: 25.4s
338:	learn: 0.3066068	total: 13s	remaining: 25.4s
339:	learn: 0.3056742	total: 13s	remaining: 25.3s
340:	learn: 0.3045644	total: 13.1s	remaining: 25.3s
341:	learn: 0.3035431	total: 13.1s	remaining: 25.2s
342:	learn: 0.3030097	total: 13.1s	remaining: 25.2s
343:	learn: 0.3021665	total: 13.2s	remaining: 25.1s
344:	learn: 0.3015335	total: 13.2s	remaining: 25.1s
345:	learn: 0.3007646	total: 13.2s	remaining: 25s
346:	learn: 0.2997277	total: 13.3s	remaining: 25s
347:	learn: 0.2987739	total: 13.3s	remaining: 24.9s
348:	learn: 0.2978556	total: 13.3s	remaining: 24.9s
349:	learn: 0.2972193	

489:	learn: 0.2122170	total: 18.6s	remaining: 19.4s
490:	learn: 0.2117929	total: 18.7s	remaining: 19.4s
491:	learn: 0.2112017	total: 18.7s	remaining: 19.3s
492:	learn: 0.2107988	total: 18.8s	remaining: 19.3s
493:	learn: 0.2104077	total: 18.8s	remaining: 19.2s
494:	learn: 0.2100445	total: 18.8s	remaining: 19.2s
495:	learn: 0.2096112	total: 18.9s	remaining: 19.2s
496:	learn: 0.2091994	total: 18.9s	remaining: 19.1s
497:	learn: 0.2087607	total: 18.9s	remaining: 19.1s
498:	learn: 0.2083200	total: 19s	remaining: 19s
499:	learn: 0.2078837	total: 19s	remaining: 19s
500:	learn: 0.2072970	total: 19s	remaining: 19s
501:	learn: 0.2063191	total: 19.1s	remaining: 18.9s
502:	learn: 0.2060822	total: 19.1s	remaining: 18.9s
503:	learn: 0.2055138	total: 19.1s	remaining: 18.8s
504:	learn: 0.2050833	total: 19.2s	remaining: 18.8s
505:	learn: 0.2045455	total: 19.2s	remaining: 18.8s
506:	learn: 0.2042679	total: 19.3s	remaining: 18.7s
507:	learn: 0.2038962	total: 19.3s	remaining: 18.7s
508:	learn: 0.2033313	to

650:	learn: 0.1501311	total: 25s	remaining: 13.4s
651:	learn: 0.1498614	total: 25s	remaining: 13.3s
652:	learn: 0.1497193	total: 25.1s	remaining: 13.3s
653:	learn: 0.1494834	total: 25.1s	remaining: 13.3s
654:	learn: 0.1489493	total: 25.1s	remaining: 13.2s
655:	learn: 0.1485530	total: 25.2s	remaining: 13.2s
656:	learn: 0.1483852	total: 25.2s	remaining: 13.2s
657:	learn: 0.1479712	total: 25.3s	remaining: 13.1s
658:	learn: 0.1476800	total: 25.3s	remaining: 13.1s
659:	learn: 0.1474661	total: 25.4s	remaining: 13.1s
660:	learn: 0.1470516	total: 25.4s	remaining: 13s
661:	learn: 0.1467481	total: 25.4s	remaining: 13s
662:	learn: 0.1466120	total: 25.5s	remaining: 12.9s
663:	learn: 0.1464380	total: 25.5s	remaining: 12.9s
664:	learn: 0.1460655	total: 25.6s	remaining: 12.9s
665:	learn: 0.1456977	total: 25.6s	remaining: 12.8s
666:	learn: 0.1454577	total: 25.7s	remaining: 12.8s
667:	learn: 0.1452722	total: 25.7s	remaining: 12.8s
668:	learn: 0.1449436	total: 25.7s	remaining: 12.7s
669:	learn: 0.144584

813:	learn: 0.1104512	total: 31.6s	remaining: 7.22s
814:	learn: 0.1103098	total: 31.6s	remaining: 7.18s
815:	learn: 0.1101187	total: 31.7s	remaining: 7.14s
816:	learn: 0.1099961	total: 31.7s	remaining: 7.11s
817:	learn: 0.1097382	total: 31.8s	remaining: 7.07s
818:	learn: 0.1095718	total: 31.8s	remaining: 7.03s
819:	learn: 0.1094612	total: 31.8s	remaining: 6.99s
820:	learn: 0.1093356	total: 31.9s	remaining: 6.95s
821:	learn: 0.1091788	total: 31.9s	remaining: 6.91s
822:	learn: 0.1090083	total: 31.9s	remaining: 6.87s
823:	learn: 0.1088125	total: 32s	remaining: 6.83s
824:	learn: 0.1086810	total: 32s	remaining: 6.79s
825:	learn: 0.1084987	total: 32.1s	remaining: 6.75s
826:	learn: 0.1082698	total: 32.1s	remaining: 6.71s
827:	learn: 0.1081411	total: 32.1s	remaining: 6.67s
828:	learn: 0.1080345	total: 32.2s	remaining: 6.63s
829:	learn: 0.1078472	total: 32.2s	remaining: 6.59s
830:	learn: 0.1077741	total: 32.2s	remaining: 6.55s
831:	learn: 0.1074488	total: 32.3s	remaining: 6.52s
832:	learn: 0.10

977:	learn: 0.0839857	total: 37.6s	remaining: 847ms
978:	learn: 0.0837855	total: 37.7s	remaining: 808ms
979:	learn: 0.0836360	total: 37.7s	remaining: 770ms
980:	learn: 0.0835328	total: 37.8s	remaining: 731ms
981:	learn: 0.0834149	total: 37.8s	remaining: 693ms
982:	learn: 0.0832511	total: 37.8s	remaining: 654ms
983:	learn: 0.0831118	total: 37.9s	remaining: 616ms
984:	learn: 0.0830418	total: 37.9s	remaining: 577ms
985:	learn: 0.0829277	total: 37.9s	remaining: 539ms
986:	learn: 0.0827402	total: 38s	remaining: 500ms
987:	learn: 0.0826715	total: 38s	remaining: 462ms
988:	learn: 0.0826318	total: 38s	remaining: 423ms
989:	learn: 0.0824855	total: 38.1s	remaining: 385ms
990:	learn: 0.0823869	total: 38.1s	remaining: 346ms
991:	learn: 0.0822154	total: 38.2s	remaining: 308ms
992:	learn: 0.0821310	total: 38.2s	remaining: 269ms
993:	learn: 0.0819602	total: 38.2s	remaining: 231ms
994:	learn: 0.0818610	total: 38.3s	remaining: 192ms
995:	learn: 0.0817207	total: 38.3s	remaining: 154ms
996:	learn: 0.0815

In [19]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [20]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 57.70%


In [21]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [22]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [23]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'D:\FCI_Material\First\Computer Vision\sec4\Alzheimer_s Dataset\test\NonDemented\26 (62).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
The image is classified as: NonDemented
